In [4]:
import rasterio
import os
from rasterio.windows import Window

def patchify(image_path, mask_path, patch_size, patch_step, output_image_folder, output_mask_folder):
    with rasterio.open(image_path) as src_img, rasterio.open(mask_path) as src_mask:
        width = src_img.width
        height = src_img.height
        
        if not os.path.exists(output_image_folder):
            os.makedirs(output_image_folder)
        if not os.path.exists(output_mask_folder):
            os.makedirs(output_mask_folder)
            
        patch_id = 0
        for i in range(0, width - patch_size + 1, patch_step):
            for j in range(0, height - patch_size + 1, patch_step):
                # Define window for image and mask
                window = Window(i, j, patch_size, patch_size)
                transform_img = src_img.window_transform(window)
                transform_mask = src_mask.window_transform(window)
                
                # Read patches
                img_patch = src_img.read(window=window)
                mask_patch = src_mask.read(1, window=window)  # Assuming single-band mask
                
                # Save image patch
                img_patch_filename = os.path.join(output_image_folder, f'image_patch_{patch_id}.tif')
                with rasterio.open(img_patch_filename, 'w', driver='GTiff',
                                   height=patch_size, width=patch_size,
                                   count=src_img.count, dtype=img_patch.dtype,
                                   crs=src_img.crs, transform=transform_img) as dst_img:
                    dst_img.write(img_patch)
                
                # Save mask patch
                mask_patch_filename = os.path.join(output_mask_folder, f'mask_patch_{patch_id}.tif')
                with rasterio.open(mask_patch_filename, 'w', driver='GTiff',
                                   height=patch_size, width=patch_size,
                                   count=1, dtype=mask_patch.dtype,
                                   crs=src_mask.crs, transform=transform_mask) as dst_mask:
                    dst_mask.write(mask_patch, 1)
                
                patch_id += 1

# Parameters
image_path = r"F:\datasets\raw data\papaya_mango\papaya_mango_raster.tif"
mask_path = r"F:\datasets\raw data\papaya_mango\papaya_mango_8bit.tif"
patch_size = 256  # Size of each patch (256x256)
patch_step = 128  # Step size for moving the patch window
output_image_folder = r"F:\datasets\dataset\papaya_mango\images"
output_mask_folder = r"F:\datasets\dataset\papaya_mango\masks"

patchify(image_path, mask_path, patch_size, patch_step, output_image_folder, output_mask_folder)
